## Extract RGB values from image
There are broadly three steps to find the dominant colors in an image:

Extract RGB values into three lists.
Perform k-means clustering on scaled RGB values.
Display the colors of cluster centers.
To extract RGB values, we use the imread() function of the image class of matplotlib. Empty lists, r, g and b have been initialized.

In [18]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow
from tensorflow import keras

In [19]:
batman_df = pd.read_csv('movies_plot.csv')
batman_df.head()

,Title,Plot
0,The Ballad of Cable Hogue,"Cable Hogue is isolated in the desert, awaitin..."
1,Monsters vs. Aliens,"In the far reaches of space, a planet explodes..."
2,The Bandit Queen,Zarra Montalvo is the daughter of an American ...
3,Broken Arrow,Major Vic Deakins (John Travolta) and Captain ...
4,Dolemite,Dolemite is a pimp and nightclub owner who is ...


In [20]:
batman_df.tail()

,Title,Plot
995,Unknown Island,Adventure-seeker Ted Osborne (Phillip Reed) an...
996,Boss Nigger,"Upon finding a wagon under attack by bandits, ..."
997,Secret Command,The plot involves a U.S. effort to root out Na...
998,The Monolith Monsters,"In the desert outside of San Angelo, Californi..."
999,Dick Tracy,"Dick Tracy (Morgan Conway), a supremely intell..."


In [21]:
batman_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   1000 non-null   object
 1   Plot    1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


In [22]:
# Read batman image and print dimensions
import matplotlib.image as img
batman_image = img.imread('batman.jpg')
print(batman_image.shape)

r =[]
g = []
b = []
# Store RGB values of all pixels in lists r, g and b
for row in batman_image:
    for temp_r, temp_g, temp_b in row:
        r.append(temp_r)
        g.append(temp_g)
        b.append(temp_b)

(720, 1280, 3)


## How many dominant colors?

In [23]:
batman_df.head()

,Title,Plot
0,The Ballad of Cable Hogue,"Cable Hogue is isolated in the desert, awaitin..."
1,Monsters vs. Aliens,"In the far reaches of space, a planet explodes..."
2,The Bandit Queen,Zarra Montalvo is the daughter of an American ...
3,Broken Arrow,Major Vic Deakins (John Travolta) and Captain ...
4,Dolemite,Dolemite is a pimp and nightclub owner who is ...


In [24]:
batman_df.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Length: 1000, dtype: bool

In [25]:
batman_df.describe()

,Title,Plot
count,1000,1000
unique,995,1000
top,My Cousin Rachel,"Cable Hogue is isolated in the desert, awaitin..."
freq,2,1


In [26]:
batman_df.info

<bound method DataFrame.info of                          Title  \
0    The Ballad of Cable Hogue   
1          Monsters vs. Aliens   
2             The Bandit Queen   
3                 Broken Arrow   
4                     Dolemite   
..                         ...   
995             Unknown Island   
996                Boss Nigger   
997             Secret Command   
998      The Monolith Monsters   
999                 Dick Tracy   

                                                  Plot  
0    Cable Hogue is isolated in the desert, awaitin...  
1    In the far reaches of space, a planet explodes...  
2    Zarra Montalvo is the daughter of an American ...  
3    Major Vic Deakins (John Travolta) and Captain ...  
4    Dolemite is a pimp and nightclub owner who is ...  
..                                                 ...  
995  Adventure-seeker Ted Osborne (Phillip Reed) an...  
996  Upon finding a wagon under attack by bandits, ...  
997  The plot involves a U.S. effort to root o

In [27]:
batman_df.describe()

,Title,Plot
count,1000,1000
unique,995,1000
top,My Cousin Rachel,"Cable Hogue is isolated in the desert, awaitin..."
freq,2,1


In [30]:
batman_df.duplicated().sum()

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Length: 1000, dtype: bool

In [28]:
batman_df.corr()

""


In [13]:
batman_df.isnull

<bound method DataFrame.isnull of                          Title  \
0    The Ballad of Cable Hogue   
1          Monsters vs. Aliens   
2             The Bandit Queen   
3                 Broken Arrow   
4                     Dolemite   
..                         ...   
995             Unknown Island   
996                Boss Nigger   
997             Secret Command   
998      The Monolith Monsters   
999                 Dick Tracy   

                                                  Plot  
0    Cable Hogue is isolated in the desert, awaitin...  
1    In the far reaches of space, a planet explodes...  
2    Zarra Montalvo is the daughter of an American ...  
3    Major Vic Deakins (John Travolta) and Captain ...  
4    Dolemite is a pimp and nightclub owner who is ...  
..                                                 ...  
995  Adventure-seeker Ted Osborne (Phillip Reed) an...  
996  Upon finding a wagon under attack by bandits, ...  
997  The plot involves a U.S. effort to root

In [14]:
batman_df.isnull().sum()

Title    0
Plot     0
dtype: int64

In [1]:
pip python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "python"



In [2]:
pip install en_core_web_sm


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement en_core_web_sm (from versions: none)
ERROR: No matching distribution found for en_core_web_sm


In [1]:
import spacy

def extract_items(text, nlp):
    items = []
    doc = nlp(text)
    quantity_map = {"one": 1, "two": 2, "three": 3, "four": 4, "five": 5, "six": 6, "seven": 7, "eight": 8, "nine": 9, "ten": 10}
    i = 0
    while i < len(doc):
        if doc[i].pos_ == "NUM":
            try:
                quantity = int(doc[i].text)
            except ValueError:
                quantity = quantity_map.get(doc[i].text.lower(), 0)
            j = i + 1
            product = []
            price = None
            while j < len(doc) and (doc[j].pos_ == "NOUN" or doc[j].pos_ == "PROPN" or doc[j].pos_ == "ADJ" or (doc[j].pos_ == "NUM" and doc[j-1].text != ".")):
                if doc[j].text == "of" and j+1<len(doc) and (doc[j+1].pos_ == "NOUN" or doc[j+1].pos_ == "PROPN" or doc[j+1].pos_ == "ADJ"):
                    j += 1
                    continue
                if doc[j].text.lower() in ["rs", "rupees"]:
                    price = float(doc[j-1].text)
                    j += 1
                    continue
                product.append(doc[j].text)
                j += 1
            if product:
                item = {'quantity': quantity, 'product': ' '.join(product)}
                if price:
                    item['price'] = price
                items.append(item)
            i = j
        else:
            i += 1
    return items

nlp = spacy.load("en_core_web_sm")
text = input("Please enter your text: ")
items = extract_items(text, nlp)
for item in items:
    print(f"Quantity: {item['quantity']}, Product: {item['product']}")
    if 'price' in item:
        print(f"Price: Rs. {item['price']}")


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.